In [1]:
import polars as pl

from mfg_capabilities.config import config


pl.Config.set_tbl_rows(10)  # to set the number of rows displayed)
# pl.Config.set_tbl_cols(10) # to set the number of columns
# pl.Config.set_fmt_str_lengths(50) # to set the max string length displayed

polars.config.Config

In [2]:
# why do some of these products have two different Standard Rates?
DELBERT_COB_PRODUCTS_PATH = config.data_dir / "from_delbert" / "COB Products by Line.xls"
delbert_cob_products = (
    pl.read_excel(DELBERT_COB_PRODUCTS_PATH, sheet_name="Sheet1")
    .filter(pl.col("Code") != "Print Date:")
    .filter(pl.col("Code") != "Country Oven Bakery")
    .sort("Product")
)
display(delbert_cob_products)

Code,Product,Line,Standard Rate Cases/Min
str,str,str,f64
"""03504""","""B03504 LAYTON WHEAT SUB ROLL""","""Roll Line""",2.08
"""03505""","""B03505 WHEAT BOLILLO""","""Roll Line""",2.08
"""03506""","""B03506 WHITE BOLILLO""","""Roll Line""",2.08
"""18402""","""B18402 SGL Mint Layers""","""Iced Cake Layers""",1.8
"""18402""","""B18402 SGL Mint Layers""","""Iced Cake Layers""",1.0
…,…,…,…
"""26814""","""B26814 CINNAMON CHIP MUFFIN""","""Cake Make-Up II""",9.375
"""26814""","""B26814 CINNAMON CHIP MUFFIN""","""Cake Make-Up II""",1.0
"""26851""","""B26851 BROWNIE HERSHEY SHEET""","""Cake Make-up""",3.83


In [3]:
delbert_cob_products_count = (
    delbert_cob_products
    .group_by("Code", "Product", "Line")
    .len()
    .sort("Product", "Line")
)
display(delbert_cob_products_count)

Code,Product,Line,len
str,str,str,u32
"""03504""","""B03504 LAYTON WHEAT SUB ROLL""","""Roll Line""",1
"""03505""","""B03505 WHEAT BOLILLO""","""Roll Line""",1
"""03506""","""B03506 WHITE BOLILLO""","""Roll Line""",1
"""18402""","""B18402 SGL Mint Layers""","""Iced Cake Layers""",2
"""18417""","""B18417 Choc Pudding Cake 20oz.""","""Cake Make-Up II""",2
…,…,…,…
"""26810""","""B26810 Variety Muffins (BB/CC)""","""Iced Cake 1/4 Sheet""",1
"""26810""","""B26810 Variety Muffins (BB/CC)""","""Iced Cake Layers""",2
"""26814""","""B26814 CINNAMON CHIP MUFFIN""","""Cake Make-Up II""",2


In [4]:
lines = (
    delbert_cob_products
    .select("Line")
    .unique()
    .sort("Line")
)
display(lines)

Line
str
"""Bread Line"""
"""Cake Make-Up II"""
"""Cake Make-up"""
"""Cookie Line"""
"""Danish Line"""
"""Iced Cake 1/4 Sheet"""
"""Iced Cake Layers"""
"""Roll Line"""


In [5]:
DELBERT_COB_KDT_PATH = PROJECT_ROOT / "data" / "from_delbert" / "YTD Downtimes with Timestamps and comments_report_7.xls"
DOWNTIME_DATA_SHEET = "Data"
delbert_cob_downtime = pl.read_excel(DELBERT_COB_KDT_PATH, sheet_name=DOWNTIME_DATA_SHEET)
display(delbert_cob_downtime)

Line,Prod Date,Equipment,Equipment Instance,Downtime Description,Start Time,End Time,Product,Operator,Min.,Notes,Maintenance Type
str,date,str,str,str,datetime[ms],datetime[ms],str,str,f64,str,str
"""Bread Line""",2025-02-02,"""General""","""General""","""Not Scheduled""",2025-02-02 04:30:00,2025-02-02 16:30:00,"""B19180 SWEET ITALIAN STIX""","""MIRANDA, GERSON""",720.0,null,"""Not Scheduled"""
"""Bread Line""",2025-02-02,"""General""","""General""","""Not Scheduled""",2025-02-02 16:30:00,2025-02-03 04:30:00,"""B19180 SWEET ITALIAN STIX""","""MIRANDA, GERSON""",720.0,null,"""Not Scheduled"""
"""Bread Line""",2025-02-03,"""General""","""General""","""Setup""",2025-02-03 04:30:00,2025-02-03 04:44:44,"""B19180 SWEET ITALIAN STIX""","""MIRANDA, GERSON""",14.733333,null,"""Planned Downtime"""
"""Bread Line""",2025-02-03,"""General""","""General""","""Setup""",2025-02-03 04:44:44,2025-02-03 05:05:34,"""B19011 ITALIAN BREAD""","""MIRANDA, GERSON""",20.833333,null,"""Planned Downtime"""
"""Bread Line""",2025-02-03,"""Bread/Roll Mixer #2""","""Bread/Roll Mixer #2""","""Mechanical Failure""",2025-02-03 05:12:24,2025-02-03 05:22:52,"""B19011 ITALIAN BREAD""","""MIRANDA, GERSON""",10.466667,null,"""Not Scheduled"""
…,…,…,…,…,…,…,…,…,…,…,…
"""Roll Line""",2025-05-15,"""General""","""General""","""Not Scheduled""",2025-05-15 16:30:00,2025-05-16 04:30:00,"""B03505 WHEAT BOLILLO""","""Non-User""",720.0,null,"""Not Scheduled"""
"""Roll Line""",2025-05-16,"""General""","""General""","""Not Scheduled""",2025-05-16 04:30:00,2025-05-16 16:30:00,"""B03505 WHEAT BOLILLO""","""HILLIS , SHELLY""",720.0,null,"""Not Scheduled"""
"""Roll Line""",2025-05-16,"""General""","""General""","""Not Scheduled""",2025-05-16 16:30:00,2025-05-17 04:30:00,"""B03505 WHEAT BOLILLO""","""HILLIS , SHELLY""",720.0,null,"""Not Scheduled"""


In [6]:
line_product = (
    delbert_cob_downtime
    .group_by("Line", "Product")
    .agg(pl.len())
    .filter(pl.col("Product") != "NONE")
    .sort("Line", "Product")
)
display(line_product)

Line,Product,len
str,str,u32
"""Bread Line""","""B19011 ITALIAN BREAD""",422
"""Bread Line""","""B19017 COUNTRY FRENCH""",255
"""Bread Line""","""B19019 SEEDED RYE BREAD""",38
"""Bread Line""","""B19068 PUMPERNICKEL BREAD""",52
"""Bread Line""","""B19100 FRENCH STIX""",11
…,…,…
"""Iced Cake Layers""","""B19839 BASE ICED WHITE""",14
"""Iced Cake Layers""","""B26810 Variety Muffins (BB/CC)""",4
"""Roll Line""","""B03505 WHEAT BOLILLO""",263


In [7]:
line_product_equip = (
    delbert_cob_downtime
    .group_by("Line", "Product", "Equipment Instance")
    .agg(pl.len())
    .sort("Line", "Product", "Equipment Instance")
)
display(line_product_equip)

Line,Product,Equipment Instance,len
str,str,str,u32
"""Bread Line""","""B19011 ITALIAN BREAD""","""Bread Bag Inserter""",1
"""Bread Line""","""B19011 ITALIAN BREAD""","""Bread Changeover""",4
"""Bread Line""","""B19011 ITALIAN BREAD""","""Bread Chunker""",7
"""Bread Line""","""B19011 ITALIAN BREAD""","""Bread Divider""",16
"""Bread Line""","""B19011 ITALIAN BREAD""","""Bread Moulder""",3
…,…,…,…
"""Roll Line""","""B19209 YEASTY""","""Roll Sheeter""",6
"""Roll Line""","""B19209 YEASTY""","""Spiral Freezer""",2
"""Roll Line""","""B19209 YEASTY""","""Sugar System""",1


In [10]:
# DOWNTIME_BY_LINE_SHEET = "Downtime by line and equipment" # pivot table in excel
# delbert_downtime_by_line_sheet = pl.read_excel(
#     DELBERT_COB_KDT_PATH,
#     sheet_name=DOWNTIME_BY_LINE_SHEET,
# )
# display(delbert_downtime_by_line_sheet)

In [11]:
# EQUIPMENT_BY_LINE_SHEET = "KDT equipment by line" # pivot table in excel
# delbert_equipment_by_line_sheet = pl.read_excel(
#     DELBERT_COB_KDT_PATH,
#     sheet_name=EQUIPMENT_BY_LINE_SHEET,
# )
# display(delbert_equipment_by_line_sheet)